<a href="https://colab.research.google.com/github/pszemraj/ai-msgbot/blob/main/notebooks/colab-notebooks/hyperparameters/gridsearch_GPT_params.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Generation Chatbot: Grid Search for Parameters

- optimal "text bot" hyperparamters are not really known, nor is there a right answer (_i.e. there is no "ground truth" dataset of 'reasonable responses to X prompt'_)
- proposal: iterate through "reasonable" parameter ranges, record output, compute sentence coherence scoring on outputs
- evaluate results once finished

In [ ]:
!pip install -U -q pandas
!pip uninstall -y -q pyarrow
!pip install -U -q pyarrow
!pip install -U -q openpyxl
import pandas as pd

     |████████████████████████████████| 11.3 MB 28.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.
     |████████████████████████████████| 25.6 MB 73.3 MB/s 
     |████████████████████████████████| 242 kB 35.5 MB/s 


## formatting

In [ ]:
from IPython.display import HTML, display
# colab formatting
def set_css():
    display(
        HTML(
            """
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  """
        )
    )

get_ipython().events.register("pre_run_cell", set_css)

# setup

In [ ]:
# !pip3 install -q torch==1.9.1+cpu  -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q torch

In [ ]:
%%capture
!pip install -U tqdm
!pip install clean-text
!pip install -q torchtext

import argparse
import gc
import os
import pprint as pp
import time
import warnings
from datetime import datetime
from os.path import join


from tqdm.auto import tqdm
from cleantext import clean
warnings.filterwarnings(action="ignore", message=".*gradient_checkpointing*")

In [ ]:
!pip install -q aitextgen

import logging

logging.basicConfig(
    format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

from aitextgen import aitextgen
from aitextgen.colab import mount_gdrive, copy_file_from_gdrive

     |████████████████████████████████| 572 kB 37.7 MB/s 
     |████████████████████████████████| 3.1 MB 43.5 MB/s 
     |████████████████████████████████| 87 kB 5.9 MB/s 
     |████████████████████████████████| 524 kB 53.7 MB/s 
     |████████████████████████████████| 132 kB 72.9 MB/s 
     |████████████████████████████████| 829 kB 31.3 MB/s 
     |████████████████████████████████| 596 kB 59.1 MB/s 
     |████████████████████████████████| 329 kB 70.0 MB/s 
     |████████████████████████████████| 1.1 MB 69.9 MB/s 
     |████████████████████████████████| 3.3 MB 65.0 MB/s 
     |████████████████████████████████| 895 kB 34.1 MB/s 
     |████████████████████████████████| 59 kB 6.2 MB/s 
     |████████████████████████████████| 160 kB 64.1 MB/s 
     |████████████████████████████████| 192 kB 73.0 MB/s 
     |████████████████████████████████| 271 kB 71.4 MB/s 


# gdrive

In [ ]:
mount_gdrive()


Mounted at /content/drive


# functions

## query_gpt_peter

In [ ]:
# should inherit main ai object
def query_gpt_peter(
                    prompt_msg: str,
                    speaker="persona alpha",
                    responder="person beta",
                    kparam=100,
                    temp=0.7,
                    nuc_range=0.8,
                    verbose=False,
                    use_gpu=False,  
                    ):


    p_list = []
    if speaker is not None:
        p_list.append(speaker.lower() + ":" + "\n")  # write prompt as the speaker
    p_list.append(prompt_msg.lower() + "\n")
    p_list.append("\n")
    p_list.append(responder.lower() + ":" + "\n")
    this_prompt = "".join(p_list)
    if verbose:
        print("overall prompt:\n")
        pp.pprint(this_prompt, indent=4)
    this_result = ai.generate(
        n=1,
        top_k=kparam,
        batch_size=512,
        max_length=128,
        min_length=16,
        prompt=this_prompt,
        temperature=temp,
        top_p=nuc_range,
        do_sample=True,
        return_as_list=True,
    )

    try:
        this_result = str(this_result[0]).split("\n")
        res_out = [str(ele).strip() for ele in this_result]
        p_out = [str(ele).strip() for ele in p_list]
        diff_list = list(
            set(res_out).difference(p_out)
        )  # remove prior prompts for the response
        this_result = [
            str(msg)
            for msg in diff_list
            if (":" not in str(msg))
            and ("szemr" not in str(msg))
            and ("peter" not in str(msg))
        ]  # remove all names
        if not isinstance(this_result, list):
            list(this_result)
        output = str(this_result[0]).strip()
        # add second line of output if first is too short (subjective)
        if len(output) < 15 and len(this_result) > 1:
            output = output + " " + str(this_result[1]).strip()
    except:
        output = "<bro, there was an error. try again>"

    p_list.append(output + "\n")
    p_list.append("\n")

    model_responses = {"out_text": output, "full_conv": p_list}

    return model_responses


## generic

In [ ]:
from IPython.display import clear_output

# clears cells in jupyter 

def isnotebook():
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'Shell':
            return True  # Colab
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter

def clear_jupyter_cell():
    is_jupyter = isnotebook()

    if is_jupyter:
        clear_output(wait=True)
    else:
        print("not in a jupyter notebook")

In [ ]:

def create_folder(directory):
    os.makedirs(directory, exist_ok=True)


def shorten_title(title_text, max_no=35):
    if len(title_text) < max_no:
        return title_text
    else:
        return title_text[:max_no] + "..."


def cleantxt_wrap(ugly_text):
    # a wrapper for clean text with options different than default

    # https://pypi.org/project/clean-text/
    cleaned_text = clean(
        ugly_text,
        fix_unicode=True,  # fix various unicode errors
        to_ascii=True,  # transliterate to closest ASCII representation
        lower=True,  # lowercase text
        no_line_breaks=True,  # fully strip line breaks as opposed to only normalizing them
        no_urls=True,  # replace all URLs with a special token
        no_emails=True,  # replace all email addresses with a special token
        no_phone_numbers=True,  # replace all phone numbers with a special token
        no_numbers=False,  # replace all numbers with a special token
        no_digits=False,  # replace all digits with a special token
        no_currency_symbols=True,  # replace all currency symbols with a special token
        no_punct=True,  # remove punctuations
        replace_with_punct="",  # instead of removing punctuations you may replace them
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="<NUM>",
        replace_with_digit="0",
        replace_with_currency_symbol="<CUR>",
        lang="en",  # set to 'de' for German special handling
    )

    return cleaned_text

### dropbox 


- [tutorial](https://python.plainenglish.io/automate-your-pdf-upload-to-dropbox-python-script-bdacc2c721f6)
- [api docs](https://dropbox-sdk-python.readthedocs.io/en/latest/api/dropbox.html?highlight=files_upload#dropbox.dropbox_client.Dropbox.files_upload)

In [ ]:
dropbox_subfolder = "GPT text gen - gridsearch - 1pt3B WoW_DD"  # @param {type:"string"}

In [ ]:
!pip install -q dropbox
import dropbox

     |████████████████████████████████| 580 kB 27.4 MB/s 
     |████████████████████████████████| 160 kB 69.1 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 


In [ ]:
token = "create-your-own"

dbx = dropbox.Dropbox(token)
pp.pprint(dbx.users_get_current_account(), compact=True, indent=4)

12/01/2021 04:31:21 — INFO — dropbox — Request to users/get_current_account


FullAccount(account_id='dbid:AAAYVJX_n2JYa7yh6anfANmwSJ7qkr-u8Do', account_type=AccountType('pro', None), country='CH', disabled=False, email='peterszemraj@gmail.com', email_verified=True, is_paired=False, locale='en', name=Name(abbreviated_name='PS', display_name='Peter Szemraj', familiar_name='Peter', given_name='Peter', surname='Szemraj'), profile_photo_url='https://dl-web.dropbox.com/account_photo/get/dbaphid%3AAACl3aYwbYAysBHXpGdGBXZj6IYsXocikxc?size=128x128&vers=1628569648991', referral_link='https://www.dropbox.com/referrals/AAAVOEoeLmg4Nw7hRIA9Wsn8OMf2_EdGbik?src=app9-10920512', root_info=UserRootInfo(home_namespace_id='9560958', root_namespace_id='9560958'), team=NOT_SET, team_member_id=NOT_SET)


In [ ]:
def get_size_mb(path2file, verbose=False):

    file_stats = os.stat(path2file)

    file_size_mb = {file_stats.st_size / (1024 * 1024)}
    if verbose: print(f'File Size in MegaBytes is {file_size_mb}')
    return round(list(file_size_mb)[0],2) # returns rounded to 2 decimals

In [ ]:
from os.path import join, basename, dirname
import time, random
from google.colab import files

def put_in_dropbox(
    vm_path, subfolder=dropbox_subfolder, no_printout=True, ncalls=0, max_calls=3
):
    if ncalls > max_calls:
        return "failed saving to DropBox - {} tries".format(ncalls)
    elif get_size_mb(vm_path) > 155:
        files.download(vm_path)
        return "file size of {} too big to put in DB, downloading".format(basename(vm_path))
    # for an item on the colab machine on path, upload to dropbox app folder at
    # subfolder/"filename"
    base_filename = basename(vm_path)
    db_path = "/{}/{}".format(subfolder, base_filename)
    try:
        with open(vm_path, "rb") as f:
            dbx.files_upload(f.read(), path=db_path, autorename=True, mute=no_printout)
    except:
        print(
            "WARNING - unable to post in dropbox, retry no. {} - ".format(ncalls + 1),
            datetime.now(),
        )
        time.sleep(random.randint(1, 3))  # small delay before trying again
        put_in_dropbox(vm_path, ncalls=ncalls + 1)  # recursion for retrying

# load model file


- <font color="orange"> IMPORTANT: while not validated _at the time of writing_, it is somewhat important to consider what optimal hyperparameter behavior _means_


In [ ]:
model_size = "355M" #@param ["355M", "774M"]
load_from_folder = True #@param {type:"boolean"}
load_folder_dir = "/content/drive/MyDrive/Programming/ai-msgbot/GPTneo_conv_33kWoW_18kDD" #@param {type:"string"}
do_grad_chkpt = False #@param {type:"boolean"}
use_gpu = False #@param {type:"boolean"}


In [ ]:
if load_from_folder:
    ai = aitextgen(model_folder=load_folder_dir, to_gpu=use_gpu,
                   gradient_checkpointing=do_grad_chkpt)
else:
    # load new 
    ai = aitextgen(tf_gpt2=model_size, to_gpu=use_gpu,
                   gradient_checkpointing=do_grad_chkpt)


12/01/2021 04:31:22 — INFO — aitextgen — Loading model from provided weights and config in //content/drive/MyDrive/Programming/ai-msgbot/GPTneo_conv_33kWoW_18kDD.
12/01/2021 04:33:19 — INFO — aitextgen — GPTNeo loaded with 1315M parameters.
12/01/2021 04:33:19 — INFO — aitextgen — Using the default GPT-2 Tokenizer.


In [ ]:
dropbox_subfolder = dropbox_subfolder + "_" + os.path.basename(load_folder_dir)

# go through loop


In [ ]:
temp_srch = list(range(50, 100, 2))
temp_srch = [float(item)/100 for item in temp_srch]
top_p_srch = list(range(50, 100,2)) 
top_p_srch = [float(item)/100 for item in top_p_srch]
topk_srch = list(range(25, 200, 25))
estimate_comp = len(temp_srch)*len(top_p_srch)*len(topk_srch)
print("anticipate approx {} calculations\n\n".format(estimate_comp))

grid_df = pd.DataFrame(columns=["temp", "topk", "top_p", "prompt", "speaker",
                                "model_response"])

grid_df.info()

anticipate approx 4375 calculations


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   temp            0 non-null      object
 1   topk            0 non-null      object
 2   top_p           0 non-null      object
 3   prompt          0 non-null      object
 4   speaker         0 non-null      object
 5   model_response  0 non-null      object
dtypes: object(6)
memory usage: 0.0+ bytes


**question lists:**

1. [medium - 40 small talk q's](https://medium.com/twyla-ai/40-small-talk-questions-your-chatbot-needs-to-know-and-why-it-matters-63caf03347f6)

In [ ]:
import random
random = random.SystemRandom() # big boy random

n = 3
print(f"\nrandom {n} values from temp list are \n", random.sample(temp_srch, k=n))
print(f"\nrandom {n} values from top_p list are \n", random.sample(top_p_srch, k=n))
print(f"\nrandom {n} values from topk list are \n", random.sample(topk_srch, k=n))


# shuffle lists for search. going thru in ~random~ order enables being able
# to somewhat look at the results during

random.shuffle(temp_srch)
random.shuffle(top_p_srch)

# a series of general questions that users may ask to test model response
generic_openers = [
                    'How are you?',
                    'What’s up?',
                    'Good morning',
                    'Tell me something',
                    'Ok',
                    'Yes',
                    'I’ll do that now',
                    'Hello',
                    'Thank you',
                    'Goodbye',
                    'How can you help me?',
                    'what can you do?',
                    'Hi, my name is……',
                    'Happy birthday!',
                    'I have a question, can you help me?',
                    'Do you know a joke?',
                    'Do you love me?',
                    'I love you',
                    'Will you marry me?',
                    'Are you single?',
                    'Do you like people?',
                    'Does Santa Claus exist?',
                    'Are you part of the Matrix?',
                    'You’re cute.',
                    'Do you have a hobby?',
                    'You’re clever',
                    'Tell me about your personality',
                    'You’re annoying',
                    'you suck',
                    'I want to speak to a human now.',
                    'Don’t you speak English?!',
                    'I want the answer NOW!',
                    'Are you human?',
                    'Are you a robot?',
                    'What is your name?',
                    'How old are you?',
                    'What’s your age?',
                    'What day is it today?',
                    'Who made you?',
                    'Which languages can you speak?',
                    'What is your mother’s name?',
                    'Where do you live?',
                    'What’s the weather like today?',
                    'Are you expensive?',
                    'Do you get smarter?',
                    'how old am I?',
                    'Did I win the lottery last night?',
                    'Will it fit in my shipping container?'
                    'Can I ask the same question in different ways and will you understand all of them correctly?',
                    'Can you tell me about yourself?'
                   
                ]

test_speakers = [
                 "emilie szemraj",
                 "michael lebens",
                 'jonas',
                 "jonathan",
                 "joost",
                 "kevin",
                 "peter",
                 "caio",
                 "arnold schwarzenegger",
                 "person alpha", # some datasets use this in the script directly
]


random 3 values from temp list are 
 [0.62, 0.92, 0.56]

random 3 values from top_p list are 
 [0.74, 0.98, 0.68]

random 3 values from topk list are 
 [100, 75, 50]


conversation starters taken from [this parade article](https://parade.com/969981/parade/conversation-starters/)

In [ ]:
pbar = tqdm(total=estimate_comp, desc="completing grid search...")

for this_temp in temp_srch:
    ind = temp_srch.index(this_temp)

    if ind > 0 and ind % 5 == 0:
        # save progress
        timestamp = datetime.now().strftime("%b-%d-%Y_%H-%M")
        grid_df.reset_index(drop=True, inplace=True)
        out_xl = "GPT-gridsearch-{}.xlsx".format(timestamp)
        out_ftr = "GPT-gridsearch-{}.ftr".format(timestamp)
        grid_df.to_excel(out_xl)
        grid_df.to_feather(out_ftr)
        put_in_dropbox(out_xl)
        put_in_dropbox(out_ftr)

    for this_top_p in top_p_srch:
        for this_topk in topk_srch:

            tst_spkr = random.sample(test_speakers, k=1)[0]
            tst_prompt = random.sample(generic_openers, k=1)[0]

            model_out = query_gpt_peter(
                                        prompt_msg=tst_prompt,
                                        speaker=tst_spkr,
                                        kparam=this_topk,
                                        temp=this_temp,
                                        nuc_range=this_top_p,
                                        verbose=False,
                                        )
            this_resp = model_out["out_text"]
            df_ind = len(grid_df)
            # ["temp", "topk", "top_p", "prompt", "speaker", "model_response"]
            grid_df.loc[df_ind] = [this_temp, this_topk, 
                                   this_top_p, tst_prompt,
                                   tst_spkr, this_resp]
            pbar.update(1)

pbar.close()

completing grid search...:   0%|          | 0/4375 [00:00<?, ?it/s]

12/01/2021 08:59:17 — INFO — dropbox — Request to files/upload
12/01/2021 08:59:17 — INFO — dropbox — Request to files/upload
12/01/2021 13:28:00 — INFO — dropbox — Request to files/upload
12/01/2021 13:28:01 — INFO — dropbox — Request to files/upload


## save final results

In [ ]:
timestamp = datetime.now().strftime("%b-%d-%Y_%H-%M")
grid_df.reset_index(drop=True, inplace=True)
out_xl = "GPT-Peter774M-FINAL-gridsearch-{}.xlsx".format(timestamp)
out_ftr = "GPT-Peter774M-FINAL-gridsearch-{}.ftr".format(timestamp)
grid_df.to_excel(out_xl)
grid_df.to_feather(out_ftr)
put_in_dropbox(out_xl)
put_in_dropbox(out_ftr)

# Analysis

In [ ]:
error_string = "<bro, there was an error. try again>"

df_analysis = grid_df.copy().convert_dtypes()
df_analysis = df_analysis[df_analysis["model_response"] != error_string]
df_analysis.reset_index(drop=True, inplace=True)
df_analysis.info(verbose=True) # overview

assume the outputs are a single sentence and then use a [sentence-scoring algorithm on them](https://github.com/simonepri/lm-scorer) via `lm-scorer` package.

- [here](https://colab.research.google.com/github/simonepri/lm-scorer/blob/master/examples/lm_scorer.ipynb) is an example notebook

In [ ]:
!pip install -q lm-scorer

import torch
from lm_scorer.models.auto import AutoLMScorer as LMScorer

# Available models
list(LMScorer.supported_model_names())
# => ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl", distilgpt2"]

# Load model to cpu or cuda
device = "cuda:0" if torch.cuda.is_available() else "cpu"
batch_size = 1
scorer = LMScorer.from_pretrained("gpt2-large", device=device,
                                  batch_size=batch_size)
print("loaded - ", datetime.now())

In [ ]:
# example

# Compute sentence score as the geometric mean of tokens' probabilities
scorer.sentence_score("I like this package.", reduce="gmean")

In [ ]:
# create helper fn

def score_text_blob(gen_text: str, method="gmean", verbose=False):
    # for method 'mean', 
    try:
        return scorer.sentence_score(gen_text, reduce=method)
    except:
        if verbose: print(f"failed computing scores for {gen_text}")
        return 0 # score sentences that failed at 0 - assumption = too undecipherable


### apply and see results

In [ ]:
df_analysis.head()

In [ ]:
import time
st = time.time()
df_analysis["model_response"] = df_analysis["model_response"].apply(str)
df_analysis["model_response"] = df_analysis["model_response"].apply(cleantxt_wrap)
df_analysis["geom_score"] = df_analysis["model_response"].apply(score_text_blob)
df_analysis["prod_score"] = df_analysis["model_response"].apply(score_text_blob, 
                                                                args=("prod",))
df_analysis["mean_score"] = df_analysis["model_response"].apply(score_text_blob, 
                                                                args=("mean",))
df_analysis["hmean_score"] = df_analysis["model_response"].apply(score_text_blob, 
                                                                args=("hmean",))
rt = (time.time() - st) / 60 # minutes
print(f"the total runtime was {rt} minutes")

## basic - plotly

- [plotly express docs](https://plotly.com/python/plotly-express/)

In [ ]:
import plotly.express as px

fig_temp = px.histogram(df_analysis, x="mean_score", color="topk",
                        marginal="box", # or violin, rug
                        hover_data=df_analysis.columns, template="plotly_dark",
                        title="Text Gen Mean Score Dist - various top_k values",
                        height=720, width=int(720*1.613))
fig_temp.show()

In [ ]:
fig_temp = px.histogram(df_analysis, x="prod_score", color="topk",
                        marginal="box", # or violin, rug
                        hover_data=df_analysis.columns, template="ggplot2",
                        title="Product Score Dist for different top_k values",
                        log_y=True, 
                        height=720, width=int(720*1.613))
fig_temp.show()

In [ ]:
df_analysis.columns

In [ ]:
px.scatter_matrix(df_analysis, template="plotly_dark", 
                    dimensions=['temp', 'topk', 'top_p','geom_score', 
                                'mean_score', 'hmean_score', 'prod_score'],
                    height=1080, width=int(1080*1.613),
                    title="scatter matrix - all quant vars")


## autoviz


In [ ]:
save_av = True #@param {type:"boolean"}


In [ ]:
%%capture
!pip install -U autoviz

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class

AV = AutoViz_Class()
filename = "for_autoviz.csv"

df_analysis.to_csv(filename, index=False)

In [ ]:
sep = ","
dft = AV.AutoViz(
    filename,
    sep=",",
    depVar="prod_score",
    dfte=None,
    header=0,
    verbose=2,
    lowess=True,
    chart_format="svg",
    max_rows_analyzed=150000,
    max_cols_analyzed=30,
)


In [ ]:
import shutil
from pathlib import Path
autopath = Path("/content/AutoViz_Plots")
av_arc = "GPT-Model Hyperparam Opt_autoviz_plots_{}".format(timestamp)
shutil.make_archive(av_arc, "gztar", autopath)

if save_av: put_in_dropbox(av_arc + ".tar.gz")

## sweetviz

In [ ]:
!pip install -q sweetviz
import sweetviz as sv

df_sv = df_analysis.copy().convert_dtypes()

feature_config = sv.FeatureConfig(skip=['prompt', 'speaker', 'model_response',],
                                  force_num=['temp', 'topk', 'top_p',
                                             'geom_score', 'prod_score'],
                                  )



df_sv = df_sv[['temp', 'topk', 'top_p', 'geom_score', 'prod_score']]

df_sv = df_sv.astype(float)

df_sv["model_response"] = df_analysis["model_response"].astype(str)
df_sv["model_response"] = df_sv["model_response"].astype("string")
df_sv.info()
# df_sv["topk"] = df_sv["topk"].

  astype(float)
# df_sv["topk"] = df_sv["topk"].astype(float)
# arameters are skip, force_cat, force_num and force_text. 

In [ ]:
my_report = sv.analyze(df_sv, 
                       target_feat="prod_score")

sv_eda_name = "GPT-Model Hyperparam Opt-SWEETVIZ-EDA-{}.html".format(timestamp)

my_report.show_html(filepath=sv_eda_name,
                    layout='vertical',) # Default arguments will generate to "SWEE

In [ ]:


from IPython.display import HTML
HTML(sv_eda_name)

In [ ]:
put_in_dropbox(sv_eda_name)

## data table + export 

In [ ]:
from google.colab import *


data_table.DataTable(df_analysis, num_rows_per_page=10)

In [ ]:
df_analysis.reset_index(drop=True, inplace=True)
data_header = "GPT-Model Hyperparam Analysis w Metrics - {}".format(timestamp)
df_analysis.to_excel(data_header + ".xlsx")
put_in_dropbox(data_header + ".xlsx")

df_analysis.to_feather(data_header + ".ftr")
put_in_dropbox(data_header + ".xlsx")

## pandas profiling 

In [ ]:
gc.collect()

In [ ]:
!pip uninstall -y -q pandas-profiling
!pip install -U -q pandas-profiling

import numpy as np
from pandas_profiling import ProfileReport

In [ ]:
gc.collect()

In [ ]:
df_profile = df_analysis[['temp', 'topk', 'top_p',
                          'prod_score', 'mean_score',
                          'hmean_score', 'model_response']]
profile = ProfileReport(df_profile, dark_mode=True,
                        title="GPT-Model Hyperparam Opt (Sentence Score)",
                        minimal=True,
                        )


profile.to_notebook_iframe()

In [ ]:

pd_eda_name = "GPT-Model Hyperparam Opt-EDA-{}.html".format(timestamp)
profile.to_file(pd_eda_name)

In [ ]:
put_in_dropbox(pd_eda_name)
print("saved - ", datetime.now())